In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from scipy import stats

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/value_models_combined.csv')
nfl_rolling = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')

In [3]:
# Want to adjust the qb rating to take into account how good the opposing defense they faced was
# Example: Two qbs have an epa of 10 in a game. But one qb played a team that normally gives up 20 epa
# The other qb played a team that normally gives up 0 epa. By the current model, both qbs had equivalent games, 
# but in reality the qb that played a tougher defense had the better performance when defense is taken into account

In [4]:
qb = nfl[['season', 'week', 'team', 'opponent', 'team_full', 'opponent_full', 'score', 'opponent_score', 
          'qb', 'qb_value']]
opposing_def = nfl_rolling[['season', 'week', 'team_full', 
          'qb_def_value']]

In [5]:
df = qb.merge(opposing_def, left_on=['season', 'week', 'opponent_full'], 
             right_on=['season', 'week', 'team_full'], suffixes = ('','_opponent'))
df = df.drop(columns=['team_full_opponent'])

In [6]:
df

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value
0,2014,1,ARI,LAC,Arizona Cardinals,Los Angeles Chargers,18,17,C. Palmer,0.491486,NaN
1,2014,1,ATL,NO,Atlanta Falcons,New Orleans Saints,37,34,M. Ryan,1.997776,NaN
2,2014,1,BAL,CIN,Baltimore Ravens,Cincinnati Bengals,16,23,J. Flacco,-0.509634,NaN
3,2014,1,BUF,CHI,Buffalo Bills,Chicago Bears,23,20,E. Manuel,-0.171272,NaN
4,2014,1,CAR,TB,Carolina Panthers,Tampa Bay Buccaneers,20,14,D. Anderson,0.728679,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4857,2022,21,KC,CIN,Kansas City Chiefs,Cincinnati Bengals,23,20,P. Mahomes,0.500521,0.509250
4858,2022,21,PHI,SF,Philadelphia Eagles,San Francisco 49ers,31,7,J. Hurts,-0.148775,0.407030
4859,2022,21,SF,PHI,San Francisco 49ers,Philadelphia Eagles,7,31,J. Johnson,-0.835878,0.398366
4860,2022,22,KC,PHI,Kansas City Chiefs,Philadelphia Eagles,38,35,P. Mahomes,1.243585,0.601635


In [7]:
# Not sure the best way to adjust ratings
# One possibility - qb_value over expected, which would be qb_value - qb_def_value. However, this is a huge adjustment
# Another possibility is a small adjustment to the qb_value based on the qb value over expected, which I think
# makes the most sense. 

# Implementing the above:
# There are going to be two adjustable thresholds
# original_value_percentage: the amount of the original value that should be kept
# adjustment_threshold: how much more value do we want to add or take away for the performance based on 
# the value over or under expected
# original value percentage of 1 and adjustment threshold of 0 means value is unchanged
# original value percentage of 0 and adjustment threshold of 1 means the value is entirely based on the value over expected
# Will likely need to mess around with these values and see what works the best 


def value_adjustment(value, adjusting_value, original_value_percentage=.95, adjustment_threshold=.1):
    value_over_expected = value + adjusting_value
    adjusted_value = (value_over_expected*adjustment_threshold) + (value*original_value_percentage)
    adjustment = adjusted_value - value
    
    return adjustment, adjusted_value

df[['qb_adjustment', 'qb_adjusted_value']] = df.apply(lambda x: value_adjustment(x.qb_value, x.qb_def_value, 
                                original_value_percentage=.7, adjustment_threshold=.3), axis=1, result_type='expand')

In [8]:
# Save df here later
df.to_csv('qb_value_modeling_adjusted_data/qb_values_with_adjustment.csv')

In [9]:
testing = df.dropna()
testing.sort_values(by='qb_adjustment', ascending=False)

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
2878,2019,8,CLE,NE,Cleveland Browns,New England Patriots,13,27,B. Mayfield,-0.928611,2.368926,0.710678,-0.217933
2866,2019,7,NYJ,NE,New York Jets,New England Patriots,0,33,S. Darnold,-3.145018,2.016382,0.604915,-2.540104
2835,2019,6,NYG,NE,New York Giants,New England Patriots,14,35,D. Jones,-1.835867,1.927344,0.578203,-1.257664
2903,2019,9,BAL,NE,Baltimore Ravens,New England Patriots,37,20,L. Jackson,1.142989,1.887971,0.566391,1.709380
53,2014,2,NE,MIN,New England Patriots,Minnesota Vikings,30,7,T. Brady,-0.093388,1.832028,0.549608,0.456221
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3771,2021,2,GB,DET,Green Bay Packers,Detroit Lions,35,17,A. Rodgers,1.459165,-1.482380,-0.444714,1.014451
3756,2021,1,SF,DET,San Francisco 49ers,Detroit Lions,41,33,J. Garoppolo,1.059005,-1.580925,-0.474278,0.584728
3794,2021,3,BAL,DET,Baltimore Ravens,Detroit Lions,19,17,L. Jackson,0.487038,-1.665859,-0.499758,-0.012719
32,2014,2,ARI,NYG,Arizona Cardinals,New York Giants,25,14,D. Stanton,-0.586498,-1.904187,-0.571256,-1.157754


In [10]:
testing[(testing.season==2022) & (testing.team_full=='San Francisco 49ers')]

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
4324,2022,1,SF,CHI,San Francisco 49ers,Chicago Bears,10,19,T. Lance,-0.713736,0.280966,0.084290,-0.629446
4356,2022,2,SF,SEA,San Francisco 49ers,Seattle Seahawks,27,7,J. Garoppolo,0.339361,-0.110239,-0.033072,0.306289
4388,2022,3,SF,DEN,San Francisco 49ers,Denver Broncos,10,11,J. Garoppolo,-1.224562,-0.194336,-0.058301,-1.282863
4420,2022,4,SF,LA,San Francisco 49ers,Los Angeles Rams,24,9,J. Garoppolo,0.835769,0.202218,0.060666,0.896434
4452,2022,5,SF,CAR,San Francisco 49ers,Carolina Panthers,37,15,J. Garoppolo,1.464973,0.003904,0.001171,1.466144
4481,2022,6,SF,ATL,San Francisco 49ers,Atlanta Falcons,14,28,J. Garoppolo,0.140460,-0.658895,-0.197669,-0.057209
4508,2022,7,SF,KC,San Francisco 49ers,Kansas City Chiefs,23,44,J. Garoppolo,0.035859,-0.194913,-0.058474,-0.022615
4537,2022,8,SF,LA,San Francisco 49ers,Los Angeles Rams,31,14,J. Garoppolo,1.182658,0.153740,0.046122,1.228780
4590,2022,10,SF,LAC,San Francisco 49ers,Los Angeles Chargers,22,16,J. Garoppolo,0.792546,0.016583,0.004975,0.797521
4619,2022,11,SF,ARI,San Francisco 49ers,Arizona Cardinals,38,10,J. Garoppolo,1.320104,0.255564,0.076669,1.396773


In [11]:
testing.sort_values(by='qb_adjusted_value', ascending=False)

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
237,2014,8,PIT,IND,Pittsburgh Steelers,Indianapolis Colts,51,34,B. Roethlisberger,3.475095,0.673309,0.201993,3.677088
3271,2020,3,KC,BAL,Kansas City Chiefs,Baltimore Ravens,34,20,P. Mahomes,3.024486,0.593677,0.178103,3.202589
2797,2019,5,HOU,ATL,Houston Texans,Atlanta Falcons,53,32,D. Watson,3.100802,-0.296399,-0.088920,3.011882
4180,2021,16,CIN,BAL,Cincinnati Bengals,Baltimore Ravens,41,21,J. Burrow,2.971832,-0.150829,-0.045249,2.926583
4498,2022,7,KC,SF,Kansas City Chiefs,San Francisco 49ers,44,23,P. Mahomes,2.524308,0.778901,0.233670,2.757978
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,2020,13,LAC,NE,Los Angeles Chargers,New England Patriots,0,45,J. Herbert,-2.814933,-0.699435,-0.209831,-3.024764
1823,2017,8,MIA,BAL,Miami Dolphins,Baltimore Ravens,0,40,M. Moore,-3.126910,0.102239,0.030672,-3.096238
4229,2021,17,NYG,CHI,New York Giants,Chicago Bears,3,29,M. Glennon,-3.402921,-0.053857,-0.016157,-3.419078
2808,2019,5,NYJ,PHI,New York Jets,Philadelphia Eagles,6,31,L. Falk,-3.372992,-0.694373,-0.208312,-3.581304


In [12]:
testing.groupby(by=['season','qb']).mean().sort_values(by='qb_value', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                                  
2017   S. Bradford         1.000000  29.000000       19.000000  1.719264   
2020   M. Mariota         15.000000  27.000000       30.000000  1.583775   
2019   M. Schaub           8.000000  20.000000       27.000000  1.421588   
2016   J. Garoppolo        1.500000  27.000000       22.500000  1.205393   
2018   P. Mahomes         10.000000  34.833333       26.166667  1.068842   
       M. Barkley         10.000000  41.000000       10.000000  1.063228   
2022   P. Mahomes         11.300000  29.200000       22.200000  0.988469   
2020   P. Mahomes         10.333333  28.944444       22.000000  0.975885   
2019   P. Mahomes         11.062500  30.500000       20.187500  0.969989   
2018   K. Allen           17.000000  33.000000       14.000000  0.932754   
2020   A. Rodgers         10.388889  31.500000       23.222222  0.914671   
2021   J. Johnson         12.500000  25.500000       43.000000  0.907915   
2016   M. Ryan            10.631579  34.105263       25.315789  0.905005   
2019   L. Jackson          9.187500  32.187500       18.750000  0.891010   
2016   T. Brady           12.933333  30.933333       16.666667  0.876759   
2014   A. Rodgers         10.705882  30.470588       21.235294  0.865107   
2020   C. Henne           17.000000  21.000000       38.000000  0.858214   
       J. Garoppolo        4.000000  27.000000       14.750000  0.814462   
2021   J. Allen           10.555556  30.888889       18.555556  0.799864   
2017   J. Garoppolo       15.000000  28.800000       19.800000  0.799191   
2018   D. Brees            9.941176  31.352941       21.176471  0.776950   
2015   L. McCown           3.000000  22.000000       27.000000  0.768115   
2021   P. Mahomes         10.950000  29.400000       22.400000  0.727618   
2014   T. Romo            10.562500  29.875000       21.375000  0.726642   
2016   A. Rodgers         10.842105  27.631579       25.052632  0.725524   
2020   J. Allen           10.473684  29.947368       23.157895  0.709459   
2018   B. Roethlisberger   9.125000  26.750000       22.500000  0.705517   
2014   T. Brady           10.882353  32.176471       19.117647  0.704528   
2020   M. Rudolph         17.000000  22.000000       24.000000  0.682109   
2016   D. Prescott         9.250000  27.437500       19.562500  0.670534   

                          qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                                 
2017   S. Bradford           -0.535180      -0.160554           1.558710  
2020   M. Mariota            -0.019556      -0.005867           1.577908  
2019   M. Schaub             -0.139806      -0.041942           1.379646  
2016   J. Garoppolo          -0.062926      -0.018878           1.186515  
2018   P. Mahomes             0.039802       0.011941           1.080782  
       M. Barkley            -0.046157      -0.013847           1.049380  
2022   P. Mahomes             0.110225       0.033068           1.021536  
2020   P. Mahomes             0.002878       0.000863           0.976749  
2019   P. Mahomes             0.006267       0.001880           0.971869  
2018   K. Allen               0.080587       0.024176           0.956930  
2020   A. Rodgers            -0.155669      -0.046701           0.867970  
2021   J. Johnson             0.037502       0.011251           0.919165  
2016   M. Ryan                0.077992       0.023398           0.928403  
2019   L. Jackson             0.147343       0.044203           0.935213  
2016   T. Brady               0.122031       0.036609           0.913368  
2014   A. Rodgers             0.224096       0.067229           0.932336  
2020   C. Henne              -0.160465      -0.048140           0.810075  
       J. Garoppolo           0.044954       0.013486           0.827948  
2021   J. Allen              -0.170283      -0.051085           0.748779  
2017   J. Garoppolo    

In [13]:
testing.groupby(by=['season', 'qb']).mean().sort_values(by='qb_adjusted_value', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                                  
2020   M. Mariota         15.000000  27.000000       30.000000  1.583775   
2017   S. Bradford         1.000000  29.000000       19.000000  1.719264   
2019   M. Schaub           8.000000  20.000000       27.000000  1.421588   
2016   J. Garoppolo        1.500000  27.000000       22.500000  1.205393   
2018   P. Mahomes         10.000000  34.833333       26.166667  1.068842   
       M. Barkley         10.000000  41.000000       10.000000  1.063228   
2015   L. McCown           3.000000  22.000000       27.000000  0.768115   
2022   P. Mahomes         11.300000  29.200000       22.200000  0.988469   
2020   P. Mahomes         10.333333  28.944444       22.000000  0.975885   
2019   P. Mahomes         11.062500  30.500000       20.187500  0.969989   
2018   K. Allen           17.000000  33.000000       14.000000  0.932754   
2019   L. Jackson          9.187500  32.187500       18.750000  0.891010   
2014   A. Rodgers         10.705882  30.470588       21.235294  0.865107   
2016   M. Ryan            10.631579  34.105263       25.315789  0.905005   
2021   J. Johnson         12.500000  25.500000       43.000000  0.907915   
2016   T. Brady           12.933333  30.933333       16.666667  0.876759   
2017   J. Garoppolo       15.000000  28.800000       19.800000  0.799191   
2020   A. Rodgers         10.388889  31.500000       23.222222  0.914671   
       J. Garoppolo        4.000000  27.000000       14.750000  0.814462   
       C. Henne           17.000000  21.000000       38.000000  0.858214   
2014   T. Brady           10.882353  32.176471       19.117647  0.704528   
2018   D. Brees            9.941176  31.352941       21.176471  0.776950   
2021   P. Mahomes         10.950000  29.400000       22.400000  0.727618   
       J. Allen           10.555556  30.888889       18.555556  0.799864   
2016   A. Rodgers         10.842105  27.631579       25.052632  0.725524   
2018   B. Roethlisberger   9.125000  26.750000       22.500000  0.705517   
2014   T. Romo            10.562500  29.875000       21.375000  0.726642   
2020   J. Allen           10.473684  29.947368       23.157895  0.709459   
2015   A. Dalton           7.000000  27.833333       16.333333  0.663245   
2017   C. Wentz            7.307692  31.076923       19.230769  0.612793   

                          qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                                 
2020   M. Mariota            -0.019556      -0.005867           1.577908  
2017   S. Bradford           -0.535180      -0.160554           1.558710  
2019   M. Schaub             -0.139806      -0.041942           1.379646  
2016   J. Garoppolo          -0.062926      -0.018878           1.186515  
2018   P. Mahomes             0.039802       0.011941           1.080782  
       M. Barkley            -0.046157      -0.013847           1.049380  
2015   L. McCown              0.865982       0.259795           1.027910  
2022   P. Mahomes             0.110225       0.033068           1.021536  
2020   P. Mahomes             0.002878       0.000863           0.976749  
2019   P. Mahomes             0.006267       0.001880           0.971869  
2018   K. Allen               0.080587       0.024176           0.956930  
2019   L. Jackson             0.147343       0.044203           0.935213  
2014   A. Rodgers             0.224096       0.067229           0.932336  
2016   M. Ryan                0.077992       0.023398           0.928403  
2021   J. Johnson             0.037502       0.011251           0.919165  
2016   T. Brady               0.122031       0.036609           0.913368  
2017   J. Garoppolo           0.297144       0.089143           0.888334  
2020   A. Rodgers            -0.155669      -0.046701           0.867970  
       J. Garoppolo           0.044954       0.013486           0.827948  
       C. Henne        

In [14]:
testing.groupby(by=['season', 'qb']).mean().sort_values(by='qb_adjustment', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                               
2019   C. McCoy         5.000000   7.000000       33.000000 -2.145695   
2020   R. Griffin      12.000000  14.000000       19.000000 -1.734426   
2021   I. Book         16.000000   3.000000       20.000000 -2.872912   
2022   T. Siemian      12.000000  10.000000       31.000000 -0.955612   
2020   R. Finley       15.000000  27.000000       17.000000 -0.251764   
       C. Streveler    17.000000   7.000000       18.000000 -1.423029   
2015   L. McCown        3.000000  22.000000       27.000000  0.768115   
2021   B. Allen        18.000000  16.000000       21.000000 -1.225659   
2014   J. Garoppolo    17.000000   9.000000       17.000000 -1.267725   
2018   S. Bradford      2.000000   6.666667       24.666667 -1.197940   
2019   R. Griffin III  17.000000  28.000000       10.000000 -0.732275   
2022   J. Stidham      17.500000  23.500000       34.000000 -0.152685   
2020   J. Driskel       2.500000  15.500000       27.000000 -0.752007   
2021   P. Walker       10.000000  34.000000       10.000000 -0.226768   
2014   R. Lindley      17.000000  13.000000       27.333333 -1.082438   
2016   C. Cook         17.500000  10.000000       25.500000 -1.871898   
       J. Brissett      3.500000  13.500000        8.000000 -0.760192   
2020   G. Gilbert       9.000000  19.000000       24.000000 -0.556204   
2017   B. Petty        16.000000  10.666667       23.666667 -0.929331   
2015   J. Clausen       8.500000  10.750000       35.750000 -1.104030   
       D. Orlovsky      5.000000  17.000000       42.000000 -0.244223   
2021   C. Rush          8.000000  20.000000       16.000000 -0.060842   
2020   J. Hurts        15.500000  20.250000       27.750000 -0.235608   
2017   B. Gabbert      13.000000  18.200000       22.800000 -1.206167   
2015   A. Luck          5.142857  22.428571       27.714286 -0.180693   
2017   E. Manuel        5.000000  17.000000       30.000000 -0.143480   
2022   J. Johnson      21.000000   7.000000       31.000000 -0.835878   
2014   D. Anderson     15.000000  19.000000       17.000000  0.435132   
2022   G. Minshew      16.500000  22.000000       30.000000 -0.332344   
2016   J. Cutler        6.400000  14.800000       24.000000 -0.788725   

                       qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                              
2019   C. McCoy            1.484363       0.445309          -1.700386  
2020   R. Griffin          1.156411       0.346923          -1.387503  
2021   I. Book             1.146971       0.344091          -2.528821  
2022   T. Siemian          1.002780       0.300834          -0.654778  
2020   R. Finley           0.962683       0.288805           0.037041  
       C. Streveler        0.907172       0.272152          -1.150877  
2015   L. McCown           0.865982       0.259795           1.027910  
2021   B. Allen            0.826635       0.247990          -0.977669  
2014   J. Garoppolo        0.792428       0.237728          -1.029997  
2018   S. Bradford         0.736034       0.220810          -0.977130  
2019   R. Griffin III      0.673627       0.202088          -0.530187  
2022   J. Stidham          0.638452       0.191536           0.038851  
2020   J. Driskel          0.629862       0.188959          -0.563049  
2021   P. Walker           0.583827       0.175148          -0.051620  
2014   R. Lindley          0.565277       0.169583          -0.912855  
2016   C. Cook             0.564024       0.169207          -1.702691  
       J. Brissett         0.521877       0.156563          -0.603629  
2020   G. Gilbert          0.515163       0.154549          -0.401655  
2017   B. Petty            0.494162       0.148249          -0.781082  
2015   J. Clausen          0.487664       0.146299          -0.957730  
       D. Orlovsky         0.486547       0.145964          -0.098259  
2021   C. Rush             0.485955